<a href="https://colab.research.google.com/github/adeakinwe/computer_vision/blob/main/MobileNet_PyTorch_5_flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchvision import models
from torchsummary import summary
import wandb
import os

In [3]:
# =======================
# STEP 0: Initialize wandb
# =======================
wandb.init(project="MobileNet-5flowers-Classifier", config={
    "epochs": 10,
    "batch_size": 16,
    "learning_rate": 0.001,
    "architecture": "MobileNet",
    "pretrained": True,
    "input_size": 224
})

# Shortcut to config values
config = wandb.config

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: adeakinwe (adeakinwe-richkinwe-io) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
# =======================
# STEP 1: Data Preparation
# =======================

# Transforms for training and validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

train_dir = "/content/drive/MyDrive/5flowersdata/flowers/train"
val_dir = "/content/drive/MyDrive/5flowersdata/flowers/val"

train_dataset = datasets.ImageFolder(root=train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(root=val_dir, transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size)

In [5]:
# ===========================
# STEP 2: Load Pretrained Model
# ===========================
from torchvision.models import MobileNet_V2_Weights

model = models.mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)

# Change the final conv layer to match 5 flower classes
model.classifier[1] = nn.Linear(model.last_channel, 5)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Unfreeze only final conv layer
for param in model.classifier[1].parameters():
    param.requires_grad = True

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Watch the model's weights and gradients
wandb.watch(model, log="all", log_freq=10)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 81.7MB/s]


In [6]:
# ===================
# STEP 3: Loss & Optimizer
# ===================

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

In [7]:
def train_model(model, criterion, optimizer, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_correct = 0
        train_total = 0
        running_loss = 0.0

        print(f"\nEpoch {epoch + 1}/{epochs}")
        print("-" * 30)

        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            batch_correct = (preds == labels).sum().item()
            train_correct += batch_correct
            train_total += labels.size(0)

            # Print every 10 batches
            if (i + 1) % 10 == 0:
                batch_acc = batch_correct / labels.size(0)
                print(f"[Batch {i+1}/{len(train_loader)}] Loss: {loss.item():.4f}, Batch Acc: {batch_acc:.4f}")

        train_acc = train_correct / train_total
        wandb.log({"epoch": epoch + 1, "train_loss": running_loss, "train_accuracy": train_acc})
        print(f"Epoch {epoch+1} Summary - Loss: {running_loss:.4f}, Train Accuracy: {train_acc:.4f}")

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = val_correct / val_total
        wandb.log({"epoch": epoch + 1, "val_accuracy": val_acc})
        print(f"Validation Accuracy: {val_acc:.4f}")


In [8]:
# ===================
# Train the model
# ===================
train_model(model, criterion, optimizer, train_loader, val_loader, epochs=config.epochs)


Epoch 1/10
------------------------------
[Batch 10/251] Loss: 1.4975, Batch Acc: 0.4375
[Batch 20/251] Loss: 1.2354, Batch Acc: 0.5000
[Batch 30/251] Loss: 1.0661, Batch Acc: 0.7500
[Batch 40/251] Loss: 1.0859, Batch Acc: 0.8125
[Batch 50/251] Loss: 0.9656, Batch Acc: 0.7500
[Batch 60/251] Loss: 0.7918, Batch Acc: 0.7500
[Batch 70/251] Loss: 1.0748, Batch Acc: 0.6250
[Batch 80/251] Loss: 0.5678, Batch Acc: 1.0000
[Batch 90/251] Loss: 1.2821, Batch Acc: 0.5625
[Batch 100/251] Loss: 0.7194, Batch Acc: 0.8125
[Batch 110/251] Loss: 0.6123, Batch Acc: 0.8750
[Batch 120/251] Loss: 0.8114, Batch Acc: 0.6875
[Batch 130/251] Loss: 0.7395, Batch Acc: 0.6875
[Batch 140/251] Loss: 0.6331, Batch Acc: 0.8750
[Batch 150/251] Loss: 0.5688, Batch Acc: 0.8750
[Batch 160/251] Loss: 0.5154, Batch Acc: 0.9375
[Batch 170/251] Loss: 0.6613, Batch Acc: 0.8125
[Batch 180/251] Loss: 0.5156, Batch Acc: 0.8750
[Batch 190/251] Loss: 0.6512, Batch Acc: 0.8125
[Batch 200/251] Loss: 0.7676, Batch Acc: 0.6250
[Batch